### Import Libs

In [105]:
import sqlite3
import random
import string

### BD Connection

In [ ]:
con = sqlite3.connect('../data/universidade_database.db')
cur = con.cursor()

### Some function to work with table data

#### Person Table

In [2]:
# randomize date to add new entry when import teacher from teacher table
def date_of_birth():
    year = random.randint(1960, 1993)
    month = random.randint(1,12)
    day = random.randint(1,30)
    date = f"{year}-{month}-{day}"
    return date

In [ ]:
# get de last id in person table to add a new entry
def next_id():
    count = cur.execute("SELECT COUNT(*) FROM person")
    amount = count.fetchall()
    last = amount[0][0]
    next_id = last+1
    return next_id

#### Teacher Table

In [ ]:
# get de last id in teacher table to add a new entry
def next_id_teacher():
    count = cur.execute("SELECT COUNT(*) FROM teacher")
    amount = count.fetchall()
    last = amount[0][0]
    next_id = last+1
    return next_id

In [4]:
# randomize a doc_number to all entries
def doc_number_teacher():
    doc = f"""{random.randint(1,10)}{random.choice(string.ascii_letters).upper()}{random.choice(string.ascii_letters).upper()}{random.randint(1,10)}{random.randint(1,10)}{random.choice(string.ascii_letters).upper()}{random.choice(string.ascii_letters).upper()}{random.choice(string.ascii_letters).upper()}{random.randint(1,10)}"""
    return doc

In [3]:
# randomize middle inital letter to name
def middle_initial():
    letter = random.choice(string.ascii_letters).upper()
    return letter

In [12]:
# Extract all teachers names from course table and put ir in a list
teachers = []
for row in cur.execute("SELECT course_id, teacher FROM course"):
    teachers.append(row[1])

In [13]:
# splited first e last name from a teacher and put both in a list of tuples
teachers_splited = []
for linha in teachers:
    teachers_splited.append(linha.split())

In [14]:
# get first name from teacher_splited list
def first_name(var):
    return teachers_splited[var][0]

In [15]:
# get last name from teacher_splited list
def last_name(var):
    return teachers_splited[var][1]

In [161]:
# quety to create a pattern to insert all teacher in teacher_splited in teacher table
def query(i):
    sqlite_insert_query = f"""INSERT INTO person VALUES ('{next_id()}', '{first_name(i)}', '{middle_initial()}', '{last_name(i)}', '{date_of_birth()}')"""
    return sqlite_insert_query

In [162]:
# execute the query above for data in teachers_splited
i=0
for nome in teachers_splited:
    count = cur.execute(query(i))
    con.commit()
    i+=1

#### Student Table

In [106]:
# randomize a doc_number for all students
def doc_number_student():
    doc = f"""{random.choice(string.ascii_letters).upper()}{random.randint(1,10)}{random.randint(1,10)}{random.choice(string.ascii_letters).upper()}{random.choice(string.ascii_letters).upper()}{random.randint(1,10)}"""
    return doc

In [102]:
# creat doc_number column in student table
cur.execute(f"ALTER TABLE student ADD doc_number")
con.commit()

In [107]:
# update doc_number in student table
for id in range(1001):
    cur.execute(f"UPDATE student SET doc_number = '{doc_number_student()}' WHERE student_id={id}")
    con.commit()

### Populating Teachers Table

In [ ]:
# list of all entries in person table. students and teachers together
persons = []
for row in cur.execute("SELECT person_id, first_name, last_name FROM person"):
    persons.append(row)

In [ ]:
# for every name in person and evey name in teacher_splited... add just teachers name's at teacher table with ID, complete name (first and last names) and doc_number
for firstname in persons:
    for name in teachers_splited:
        if firstname[1] == name[0] and firstname[2] == name[1]:
            print(f"{firstname[0]}, {name[0]}, {name[1]}")
            query_teacher = f"""INSERT INTO teacher VALUES ('{next_id_teacher()}', '{firstname[0]}', '{doc_number_teacher()}')"""
            count = cur.execute(query_teacher)
            con.commit()

### Modifying Tables

In [32]:
# geting emails from student table
emails = []
for row in cur.execute("SELECT email FROM student"):
    emails.append(row[0])

In [33]:
# replacing "@university" for "@al.university" 
new_email = []
for email in emails:
    new = email.replace("university", "al.university")
    new_email.append(new)

In [31]:
# create email column in person table
cur.execute("ALTER TABLE person ADD email")
con.commit()

In [34]:
# update student's new email format at person table (transfer email from students to person)
id = 1
id_lista = 0
for numro in new_email:
    cur.execute(f"UPDATE person SET email = '{new_email[id_lista]}' WHERE personid={id}")
    id+=1
    id_lista+=1
    con.commit()


In [18]:
# update teacher's email in person table
id = 1000 #where the teacher starts
for numero in range(31):
    cur.execute(f"UPDATE person SET email = person.firstname||person.lastname||'@prof.univertisy.com' WHERE personid={id}")
    id+=1
    con.commit()

In [74]:
# geting teacher's id from person table and put it in a list
ids = []
for row in cur.execute('SELECT t.person_id FROM person p JOIN teacher t ON p.person_id = t.teacher_id'):
    ids.append(row[0])


In [84]:
# exchanging the teacher's name for teacher's id in course table
x = 1
for id in ids:
    cur.execute(f"UPDATE course SET teacher_id = {id} WHERE course_id={x}")
    x+=1
    con.commit()

### Renaming Columns

In [54]:
cur.execute("ALTER TABLE person RENAME COLUMN person_ID TO person_id")
con.commit()

In [ ]:
cur.execute("ALTER TABLE course RENAME COLUMN name TO course_name")
con.commit()

In [52]:
cur.execute("ALTER TABLE credit RENAME COLUMN courseid TO course_id")
con.commit()

In [55]:
cur.execute("ALTER TABLE student RENAME COLUMN personid TO person_id")
con.commit()

In [58]:
cur.execute("ALTER TABLE teacher RENAME COLUMN docnumber TO doc_number")
con.commit()

### SELECTS

In [ ]:
# count entries in person table
cur.execute('SELECT COUNT(*) FROM person')
cur.fetchall()

In [ ]:
cur.execute("SELECT * FROM teacher")
cur.fetchall()

In [ ]:
cur.execute("SELECT COUNT(*) FROM person")
cur.fetchall()

In [ ]:
cur.execute("SELECT COUNT(*) FROM teacher")
cur.fetchall()

In [ ]:
cur.execute("SELECT * FROM person WHERE personid>1000")
cur.fetchall()